In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import tensorflow as tf
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')
cats = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

print('There are %d total dog breeds.' % len(cats))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog breeds.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [3]:
img_width, img_height = 299, 299
train_data_dir = "dogImages/train"
validation_data_dir = "dogImages/valid"
test_data_dir = "dogImages/test"
nb_train_samples = len(train_files)
nb_validation_samples = len(valid_files)
nb_test_samples = len(test_files)
batch_size = 20
epochs = 100
model = applications.InceptionV3(weights = "imagenet", include_top=False)
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [4]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(133, activation="softmax")(x)


model_final = Model(input = model.input, output = predictions)
for layer in model.layers:
    layer.trainable = False

model_final.compile(loss = "categorical_crossentropy", 
                    optimizer='rmsprop', metrics=["accuracy"])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [5]:
datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)
test_datagen = ImageDataGenerator(
rescale = 1./255)

In [6]:
train_generator = datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")


Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.


In [7]:
checkpoint = ModelCheckpoint("Inception.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [8]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [9]:
model_final.load_weights("Inception.h5")

In [9]:
model_final.fit_generator(
train_generator,
steps_per_epoch = nb_train_samples//batch_size,
epochs = epochs,
validation_data = validation_generator,  
validation_steps = nb_validation_samples//batch_size,
callbacks = [checkpoint, early])

Epoch 1/100
334/334 [==============================] - 182s - loss: 2.7580 - acc: 0.3662 - val_loss: 1.3299 - val_acc: 0.6142
Epoch 2/100
334/334 [==============================] - 173s - loss: 1.4854 - acc: 0.5909 - val_loss: 1.1990 - val_acc: 0.6635
Epoch 3/100
334/334 [==============================] - 174s - loss: 1.3238 - acc: 0.6269 - val_loss: 1.0585 - val_acc: 0.6976
Epoch 4/100
334/334 [==============================] - 173s - loss: 1.2827 - acc: 0.6394 - val_loss: 1.0604 - val_acc: 0.7026
Epoch 5/100
334/334 [==============================] - 173s - loss: 1.2267 - acc: 0.6573 - val_loss: 1.0750 - val_acc: 0.6952
Epoch 6/100
334/334 [==============================] - 172s - loss: 1.1735 - acc: 0.6750 - val_loss: 1.0085 - val_acc: 0.7129
Epoch 7/100
334/334 [==============================] - 172s - loss: 1.1878 - acc: 0.6734 - val_loss: 1.0054 - val_acc: 0.7124
Epoch 8/100
334/334 [==============================] - 171s - loss: 1.1516 - acc: 0.6834 - val_loss: 0.9698 - val_acc:

In [12]:
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [13]:
for layer in model.layers[:229]:
    layer.trainable = False
model_final.load_weights("Inception.h5")
for layer in model.layers[229:]:
    layer.trainable = True

In [14]:
model_final.fit_generator(
train_generator,
steps_per_epoch = nb_train_samples//batch_size,
epochs = epochs,
validation_data = validation_generator,
validation_steps = nb_validation_samples//batch_size,
callbacks = [checkpoint, early])

Epoch 1/100
334/334 [==============================] - 180s - loss: 0.9163 - acc: 0.7412 - val_loss: 0.7877 - val_acc: 0.7693
Epoch 2/100
334/334 [==============================] - 176s - loss: 0.7718 - acc: 0.7617 - val_loss: 0.7081 - val_acc: 0.7943
Epoch 3/100
334/334 [==============================] - 176s - loss: 0.7586 - acc: 0.7746 - val_loss: 0.7238 - val_acc: 0.7857
Epoch 4/100
334/334 [==============================] - 175s - loss: 0.7019 - acc: 0.7855 - val_loss: 0.7510 - val_acc: 0.7847
Epoch 5/100
334/334 [==============================] - 174s - loss: 0.6772 - acc: 0.7924 - val_loss: 0.7267 - val_acc: 0.7888
Epoch 6/100
334/334 [==============================] - 177s - loss: 0.6475 - acc: 0.7988 - val_loss: 0.7269 - val_acc: 0.7847
Epoch 7/100
334/334 [==============================] - 174s - loss: 0.6245 - acc: 0.8042 - val_loss: 0.6892 - val_acc: 0.8046
Epoch 8/100
334/334 [==============================] - 176s - loss: 0.6238 - acc: 0.8099 - val_loss: 0.7067 - val_acc:

334/334 [==============================] - 174s - loss: 0.3654 - acc: 0.8823 - val_loss: 0.5778 - val_acc: 0.8285
Epoch 34/100
334/334 [==============================] - 177s - loss: 0.3716 - acc: 0.8817 - val_loss: 0.6418 - val_acc: 0.8222
Epoch 35/100
334/334 [==============================] - 177s - loss: 0.3697 - acc: 0.8832 - val_loss: 0.6125 - val_acc: 0.8262
Epoch 36/100
334/334 [==============================] - 174s - loss: 0.3637 - acc: 0.8847 - val_loss: 0.6291 - val_acc: 0.8206
Epoch 37/100
334/334 [==============================] - 174s - loss: 0.3464 - acc: 0.8873 - val_loss: 0.6381 - val_acc: 0.8122
Epoch 38/100
334/334 [==============================] - 177s - loss: 0.3603 - acc: 0.8816 - val_loss: 0.6299 - val_acc: 0.8349
Epoch 39/100
334/334 [==============================] - 177s - loss: 0.3544 - acc: 0.8834 - val_loss: 0.6200 - val_acc: 0.8137
Epoch 40/100
334/334 [==============================] - 174s - loss: 0.3456 - acc: 0.8867 - val_loss: 0.6422 - val_acc: 0.81

In [17]:
test_generator = test_datagen.flow_from_directory(
test_data_dir, batch_size =1, 
target_size = (img_height, img_width),
    shuffle = "false",
class_mode = "categorical")
#names = [output.name for output in model_final.outputs]
#print(names, '\n')
#prediction = model_final.predict_generator(test_generator, steps = 836)

#print(prediction)
accuracy = model_final.evaluate_generator(test_generator, steps = 836)
print(accuracy)

Found 836 images belonging to 133 classes.
[0.35435321373837364, 0.90789473684210531]


In [19]:
print(model_final.metrics_names)

['loss', 'acc']
